In [1]:
import pandas as pd

korean_air_data = pd.read_csv("korean_air_data/korean_air_2020_07.csv")

korean_air_data.columns = ["a","title","text","editor","times","date"]

del korean_air_data["a"]

# na값 제거
data = korean_air_data.dropna()

# 불용어 지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','박정환','도','을','것','를','등','으로','자','에','등','와','한','하다''것','\n','로']

# 한글 외의 모든 데이터 제거 : 기업명이 영어로 되어있는 경우를 어떻게 처리 해야할까?
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [2]:
from math import log10
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

# t is token
# d is document == tokens
# D is documents == document list

def f(t, d):
    return d.count(t)

def tf(t, d):
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)



def tfidfScorer(D):
    tokenized_D = [tokenizer(d) for d in D]
    result = []
    for d in tokenized_D:
        result.append([(t, tfidf(t, d, tokenized_D)) for t in d])
    return result

def tokenizer(d):
    okt = Okt()
    def keyword_extractor(text):
        tokens = okt.phrases(text)
        tokens = [token for token in tokens if len(token) > 1]  # 한 글자인 단어는 제외
        count_dict = [(token, text.count(token)) for token in tokens]
        ranked_words = sorted(count_dict, key=lambda x: x[1], reverse=True)[:20]
        return [keyword for keyword, freq in ranked_words]
    return keyword_extractor(d)



In [3]:
for i, doc in enumerate(tfidfScorer(data)):
        print('====== document[%d] ======' % i)
        print(doc)

====== document[0] ======
[('사업', 0.6812412373755872), ('이익', 1.5563025007672873), ('영업이익', 1.6812412373755872), ('진행', 1.097664651741638), ('영업', 1.255272505103306), ('천억', 1.8573324964312685), ('천억 원', 1.9822712330395684), ('부문', 1.760422483423212), ('분기', 0.9030899869919435), ('하이브리드', 1.6812412373755872), ('환경', 1.9822712330395684), ('드론', 1.614294447744974), ('화학', 1.6812412373755872), ('지난', 0.7269987279362623), ('이번', 1.0791812460476249), ('친환경', 2.1583624920952498), ('에너지', 1.9822712330395684), ('하이브리드 드론', 1.760422483423212), ('온라인', 1.8573324964312685), ('햇반', 2.1583624920952498)]
====== document[1] ======
[('항공', 0.3766071174427808), ('유상증자', 1.097664651741638), ('주주', 1.180638886806402), ('제주항공', 1.345449135452394), ('자금', 1.0614524790871933), ('투자', 0.9822712330395684), ('코로나', 0.510002481114318), ('인수', 0.9542425094393249), ('업계', 0.8796088911424207), ('참여', 1.505149978319906), ('억원', 0.5149098156090622), ('중단', 1.614294447744974), ('진행', 1.097664651741638), ('일정', 2.1583

In [4]:
data

0       화학이 지난 분기 매출 조천억 원 영업이익 천억 원의 경영실적을 달성했습니다지난해...
3      제주항공 이스타 인수 중단에 한 차례 더 연기저비용 항공사들이 자금난을 해소하기 위...
4      주택담보대출채권이 증가하면서 상반기 자산유동화증권발행이 배로 늘어났다일 금융감독원이...
5      일본 양대 항공사인 전일본공수와 일본항공이 올해 분기 사상 최악인 억엔약 조억원의 ...
6      올해 상반기 자산유동화증권 발행금액이 지난해 같은 기간의 약 두 배로 불어났다특히 ...
                             ...                        
418    코로나 여파로 항공업계가 고사 위기에 처한 가운데 올해 분기 국내 항공사의 국제선 ...
422    아이티엠반도체는 종속회사인     가 국민은행으로부터 빌린 억원에 대해서 채무보증을...
423    보령제약   지분 억만원에 취득 결정 마다가스카르 암바토비 니켈 광산 및 제련소 운...
424    머니투데이 강민수 기자 종합한진칼이 이날부터 억원 규모 분리형 신주인수권부사채 공모...
425    로 배우는 영작문 로 배우는 영작문으로 영문 뉴스 읽기에 도전해보세요 영문뉴스 원문...
Name: text, Length: 288, dtype: object